In [1]:
# Importing required libraries

import time
import datetime

from bs4 import BeautifulSoup
from selenium import webdriver

from pymongo import MongoClient

In [2]:
# Creating a connection to MongoDB
client = MongoClient('localhost', 27017)
db = client['news']
collection = db['elespectador']

In [3]:
# Base URL of the site to be analyzed
SITE_URL = 'https://www.elespectador.com'

In [4]:
# Firefox web driver path
# Download the driver for you S.O. here: https://github.com/mozilla/geckodriver/releases
DRIVER_PATH = './geckodriver.exe'

In [5]:
# Creating a new firefox window
browser = webdriver.Firefox(executable_path = DRIVER_PATH)

<ipython-input-5-c8b61cbdd697>:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  browser = webdriver.Firefox(executable_path = DRIVER_PATH)


In [6]:
def make_request(browser, relative_path):
    # Making the request and rendering the browser
    browser.get(SITE_URL + relative_path)
    
    # Simulating vertical scrolling for handling lazy load
    check_height = browser.execute_script('return document.body.scrollHeight;')
    while True:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        time.sleep(3)
        height = browser.execute_script('return document.body.scrollHeight;')
        if height == check_height: 
            break 
        check_height = height
    
    # Getting HTML content and passing it to BeautifulSoup for scraping analysis
    return BeautifulSoup(browser.page_source, 'html.parser')

In [7]:
# Getting HTML content for news listing page
mayor = ['archivo']
categoria = ['politica', 'ciencia', 'salud', 'educacion', 'judicial']
paginacion = [1, 2, 3]


for m in mayor:
    for c in categoria:
        for p in paginacion:
            soup = make_request(browser, '/'+m+'/'+c+'/'+str(p)+'/')
            # Finding the section where news are contained 
            layout = soup.find(class_ = 'Layout-flexAds')
            # Getting blocks from layout
            blocks = layout.find('section').find_all(recursive = False)
            print(len(blocks)) # 3 blocks founded, 2 for news and 1 for pagintion
            # Finding and concatenating news cards
            cards = blocks[0].find_all(class_ = 'Card') + blocks[1].find_all(class_ = 'Card')
            len(cards)
            news = []

            for card in cards:
                news.append({
                    'title': card.find('h2', class_ = 'Card-Title').find('a').get_text(),
                    'relative_path': card.find('h2', class_ = 'Card-Title').find('a')['href']
                })
            
            news
            for n in news:
                # Getting HTML content for news page
                soup = make_request(browser, n['relative_path'])

                # Extracting news metadata
                
                if soup.find(class_ = 'ArticleHeader-Date'):
                    
                    var = soup.find(class_ = 'ArticleHeader-Date').get_text()
                    x = []
                    x = var.split('-')                            
                    newvar = ''.join(char for char in x[0] if char.isalnum())
                    var2 = newvar.replace("sept","sep")
                    n['datetime'] = datetime.datetime.strptime(var2, "%d%b%Y")
                    #n['datetime'] = soup.find(class_ = 'ArticleHeader-Date').get_text()
                else:
                    n['datetime'] = 'Nada'
                    
                if soup.find(class_ = 'ACredit-Author').find('a'):
                    n['author'] = soup.find(class_ = 'ACredit-Author').find('a').get_text() 
                else:
                    n['author'] = 'Nada'
                    
                if soup.find(class_ = 'ArticleHeader-Hook').find('div'):
                    n['summary'] = soup.find(class_ = 'ArticleHeader-Hook').find('div').get_text()
                else:
                    n['summary'] = 'Nada'
                

                # Extracting and concatenating news full text
                paragraphs = soup.find_all(class_ = 'font--secondary')
                n['full_text'] = ' '.join([p.get_text() for p in paragraphs])
            news
            # Storing extracted information for further analysis
            collection.insert_many(news)

2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
